# Домашнее задание


# 🎈🍾 🎉





Вы работаете в крупном онлайн-магазине, который реализует товары «всё для праздников». Вам необходимо провести ABC-анализ и выявить наиболее важные товары на основе объемов продаж и популярности. Эта информация составит план закупок на весь следующий год. 

**ABC-анализ** позволяет определить наиболее важные ресурсы компании на основе объемов продаж и прибыли.

В основе ABC-анализа лежит __«Принцип Парето»: 20% ресурсов приносят 80% прибыли__. Именно этот вид анализа позволяет выделить те самые  группы товаров, ресурсов, клиентов, поставщиков и т. п., которые и приносят компании основную прибыль.

__Задача анализа разделить товары на группы по принципу:__

- А — наиболее ценные, 20% — ассортимента (номенклатура); 80% — продаж
- В — промежуточные, 30% — ассортимента; 15% — продаж
- С — наименее ценные, 50% — ассортимента; 5% — продаж

## Задание 1

Сохраните все данные из таблицы `products` из PostgreSQL в дата-фрейм `products` и подготовьте данные к анализу.

1. Подгрузите данные для подключения
2. Подключите нужную библиотеку для работы с PostgreSQL
3. Создайте соединение с БД
4. Создайте запрос и выгрузите все данные из таблицы `products`
5. Положите данные в дата-фрейм `products`
6. Приведите данные к удобному типу: числа к числовому типу, идентификаторы — к текстовому


In [2]:
from postgresql_secret import pg_secret

In [3]:
import psycopg2

connection = psycopg2.connect(
                             database=pg_secret['db_name'],
                             user=pg_secret['db_user'],
                             password=pg_secret['db_password'],
                             host=pg_secret['db_host'],
                             port=pg_secret['db_port'],
                             )

cursor = connection.cursor()# заведи курсор, чтобы делать запросы в базу

In [4]:
import pandas as pd

query = "SELECT * FROM products" # напиши запрос, твоих знаний SQL точно хватит
cursor.execute(query)
result = cursor.fetchall()
cursor.description
columns = []
for desc in cursor.description:
    columns.append(desc[0])


products = pd.DataFrame(result, columns = columns)
products.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   invoice_id   779495 non-null  int64 
 1   stock_code   779495 non-null  object
 2   description  779495 non-null  object
 3   quantity     779495 non-null  int64 
 4   price        779495 non-null  object
dtypes: int64(2), object(3)
memory usage: 29.7+ MB


In [5]:
products['invoice_id'] = products['invoice_id'].apply(str)
products['price'] = products['price'].astype('float64')
products.head()

,invoice_id,stock_code,description,quantity,price
0,513762,21913,VINTAGE SEASIDE JIGSAW PUZZLES,4,3.75
1,513762,21914,BLUE HARMONICA IN BOX,12,1.25
2,513762,21918,SET 12 KIDS COLOUR CHALK STICKS,24,0.42
3,513762,21931,JUMBO STORAGE BAG SUKI,10,1.95
4,513762,21975,PACK OF 60 DINOSAUR CAKE CASES,24,0.55


## Задание 2

__Посчитайте, какой доход принес каждый продукт и сколько единиц каждого продукта было продано__

1. Посчитай общую сумму оплат для каждой строки счета
2. Получи общий доход и количество единиц по товарам
3. Объедини сгруппированные данные в один датафрейм со столбцами: `description`, `stock_code`, `quantity`, `total`

In [6]:
products['total'] = products['quantity']*products['price']
products.head()

,invoice_id,stock_code,description,quantity,price,total
0,513762,21913,VINTAGE SEASIDE JIGSAW PUZZLES,4,3.75,15.00
1,513762,21914,BLUE HARMONICA IN BOX,12,1.25,15.00
2,513762,21918,SET 12 KIDS COLOUR CHALK STICKS,24,0.42,10.08
3,513762,21931,JUMBO STORAGE BAG SUKI,10,1.95,19.50
4,513762,21975,PACK OF 60 DINOSAUR CAKE CASES,24,0.55,13.20


In [7]:
products_total = products.groupby(['stock_code', 'description'])['total'].sum().reset_index()
products_total
# посчитай общих доход по каждому товару

,stock_code,description,total
0,10002,INFLATABLE POLITICAL GLOBE,6638.27
1,10080,GROOVY CACTUS INFLATABLE,124.61
2,10109,BENDY COLOUR PENCILS,1.68
3,10120,DOGGY RUBBER,136.08
4,10123C,HEARTS WRAPPING TAPE,226.76
...,...,...,...
5310,PADS,PADS TO MATCH ALL CUSHIONS,0.00
5311,POST,POSTAGE,124648.04
5312,SP1002,KID'S CHALKBOARD/EASEL,14.75
5313,TEST001,This is a test product.,225.00


In [8]:
products_count = products.groupby('stock_code')['quantity'].sum().reset_index()
products_count
# посчитай общее количество проданных единиц по каждому товару

,stock_code,quantity
0,10002,8479
1,10080,303
2,10109,4
3,10120,648
4,10123C,628
...,...,...
4626,PADS,18
4627,POST,5235
4628,SP1002,5
4629,TEST001,60


In [9]:
products = pd.merge(products_total, products_count, on = 'stock_code', how = 'left')
products
# объедини общие данные в одну таблицу

,stock_code,description,total,quantity
0,10002,INFLATABLE POLITICAL GLOBE,6638.27,8479
1,10080,GROOVY CACTUS INFLATABLE,124.61,303
2,10109,BENDY COLOUR PENCILS,1.68,4
3,10120,DOGGY RUBBER,136.08,648
4,10123C,HEARTS WRAPPING TAPE,226.76,628
...,...,...,...,...
5310,PADS,PADS TO MATCH ALL CUSHIONS,0.00,18
5311,POST,POSTAGE,124648.04,5235
5312,SP1002,KID'S CHALKBOARD/EASEL,14.75,5
5313,TEST001,This is a test product.,225.00,60


## Задание 3

__Найди нарастающий итог по процентам__

1. Отсортируй датафрейм по доходу в нисходящем порядке 
2. Найди процент дохода каждого товара в общем доходе 
3. Сделай нарастающий итог (running total) по процентам каждого товара, для этого найди в интернете и примени метод для расчета кумулятивной суммы в `pandas`. 


Нарастающий итог — это представление данных, в котором мы видим динамику нарастания или убывания суммы от начала отсчета до конца. Получаем мы эту кумулятивную сумму, прибавлением поочередно каждого следующего элемента к сумме предыдущих.  



In [13]:
products.sort_values(by='total', ascending=False)
products
# отсортируй дата-фрейм (большие вверху, маленькие внизу)

,stock_code,description,total,quantity
1862,22423,REGENCY CAKESTAND 3 TIER,277656.25,24139
4751,85123A,WHITE HANGING HEART T-LIGHT HOLDER,247048.01,91814
3283,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.60,80995
5309,M,Manual,151777.67,9391
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436
...,...,...,...,...
4042,84206B,CAT W SUNGLASSES BLANK CARD,0.76,9
3015,23366,SET 12 COLOURING PENCILS DOILEY,0.65,1919
3406,35930,PINK HEART CHRISTMAS DECORATION,0.38,1
4040,84205C,HAPPY BIRTHDAY GINGER CAT CARD,0.38,2


In [15]:
products['percent_total'] = round((products['total']/round(products['total'].sum()))*100,2)
products
# найди процент дохода для каждого товара 

#(искомая часть / целое число) * 100

,stock_code,description,total,quantity,percent_total
1862,22423,REGENCY CAKESTAND 3 TIER,277656.25,24139,1.60
4751,85123A,WHITE HANGING HEART T-LIGHT HOLDER,247048.01,91814,1.42
3283,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.60,80995,0.97
5309,M,Manual,151777.67,9391,0.87
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77
...,...,...,...,...,...
4042,84206B,CAT W SUNGLASSES BLANK CARD,0.76,9,0.00
3015,23366,SET 12 COLOURING PENCILS DOILEY,0.65,1919,0.00
3406,35930,PINK HEART CHRISTMAS DECORATION,0.38,1,0.00
4040,84205C,HAPPY BIRTHDAY GINGER CAT CARD,0.38,2,0.00


In [17]:
products['running_total_percent'] = round (products['percent_total'].cumsum(),2)
products
# используй метод pandas для кумулятивной суммы

,stock_code,description,total,quantity,percent_total,running_total_percent
1862,22423,REGENCY CAKESTAND 3 TIER,277656.25,24139,1.60,1.60
4751,85123A,WHITE HANGING HEART T-LIGHT HOLDER,247048.01,91814,1.42,3.02
3283,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.60,80995,0.97,3.99
5309,M,Manual,151777.67,9391,0.87,4.86
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77,5.63
...,...,...,...,...,...,...
4042,84206B,CAT W SUNGLASSES BLANK CARD,0.76,9,0.00,97.33
3015,23366,SET 12 COLOURING PENCILS DOILEY,0.65,1919,0.00,97.33
3406,35930,PINK HEART CHRISTMAS DECORATION,0.38,1,0.00,97.33
4040,84205C,HAPPY BIRTHDAY GINGER CAT CARD,0.38,2,0.00,97.33


## Задание 4

__Пометь товары метками `'A'`, `'B'`, `'C'` в зависимости от нарастающего итога__

1. Напиши функцию, которая будет помечать товары с нарастающим итогом ниже или равно 80% как группу `'A'`, от 80%-95% — как `'B'` и остальные как `'C'`

In [19]:
def revenue(x):
    if x <=80:
        return 'A'
    if x <=95:
        return 'B'
    else:
        return 'C'
    # допиши функцию
products['revenue'] = products['running_total_percent'].apply(revenue)
products

# создай новый столбец revenue с помощью функции

,stock_code,description,total,quantity,percent_total,running_total_percent,revenue
1862,22423,REGENCY CAKESTAND 3 TIER,277656.25,24139,1.60,1.60,A
4751,85123A,WHITE HANGING HEART T-LIGHT HOLDER,247048.01,91814,1.42,3.02,A
3283,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.60,80995,0.97,3.99,A
5309,M,Manual,151777.67,9391,0.87,4.86,A
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77,5.63,A
...,...,...,...,...,...,...,...
4042,84206B,CAT W SUNGLASSES BLANK CARD,0.76,9,0.00,97.33,C
3015,23366,SET 12 COLOURING PENCILS DOILEY,0.65,1919,0.00,97.33,C
3406,35930,PINK HEART CHRISTMAS DECORATION,0.38,1,0.00,97.33,C
4040,84205C,HAPPY BIRTHDAY GINGER CAT CARD,0.38,2,0.00,97.33,C


## Задание 5

__Найди нарастающий итог по количеству товарных единиц и пометь товары метками `'A'`, `'B'`, `'C'`, отмечая тем самым популярность товара__

1. Отсортируй предыдущий датафрейм с данными о доходе теперь по количеству товаров
2. Найди процент каждого товара в общем количестве
3. Найди нарастающий итог по количеству
4. Напиши функцию с теми же параметрами, что и у дохода
5. Отметь группу

In [20]:
products = products.sort_values(by='quantity', ascending = False)
products
# проведи те же операции, что в двух предыдущих заданиях, для оценки популярности товаров

,stock_code,description,total,quantity,percent_total,running_total_percent,revenue
4031,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,24098.03,105185,0.14,29.60,A
4727,85099B,RED RETROSPOT JUMBO BAG,9289.35,93436,0.05,57.94,A
4726,85099B,JUMBO BAG RED WHITE SPOTTY,24323.85,93436,0.14,28.62,A
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77,5.63,A
4750,85123A,CREAM HANGING HEART T-LIGHT HOLDER,155.35,91814,0.00,97.33,C
...,...,...,...,...,...,...,...
855,21409,SPOTTY COCKEREL DOORSTOP,4.25,1,0.00,97.33,C
3794,72751C,"F.FAIRY S/3 SML CANDLE, ROSE",5.95,1,0.00,97.33,C
4904,90000C,OPAL WHITE/SILVER FLOWER NECKLACE,5.95,1,0.00,97.33,C
3792,72751A,"F.FAIRY S/3 SML CANDLE,LILY/VALLEY",5.95,1,0.00,97.33,C


In [21]:
products['percent_quantity']= round((products['quantity']/products['quantity'].sum())*100,2)
products

,stock_code,description,total,quantity,percent_total,running_total_percent,revenue,percent_quantity
4031,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,24098.03,105185,0.14,29.60,A,0.77
4727,85099B,RED RETROSPOT JUMBO BAG,9289.35,93436,0.05,57.94,A,0.69
4726,85099B,JUMBO BAG RED WHITE SPOTTY,24323.85,93436,0.14,28.62,A,0.69
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77,5.63,A,0.69
4750,85123A,CREAM HANGING HEART T-LIGHT HOLDER,155.35,91814,0.00,97.33,C,0.68
...,...,...,...,...,...,...,...,...
855,21409,SPOTTY COCKEREL DOORSTOP,4.25,1,0.00,97.33,C,0.00
3794,72751C,"F.FAIRY S/3 SML CANDLE, ROSE",5.95,1,0.00,97.33,C,0.00
4904,90000C,OPAL WHITE/SILVER FLOWER NECKLACE,5.95,1,0.00,97.33,C,0.00
3792,72751A,"F.FAIRY S/3 SML CANDLE,LILY/VALLEY",5.95,1,0.00,97.33,C,0.00


In [23]:
products['running_percent_quantity'] = round(products['percent_quantity'].cumsum(),2)
products

,stock_code,description,total,quantity,percent_total,running_total_percent,revenue,percent_quantity,running_percent_quantity
4031,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,24098.03,105185,0.14,29.60,A,0.77,0.77
4727,85099B,RED RETROSPOT JUMBO BAG,9289.35,93436,0.05,57.94,A,0.69,1.46
4726,85099B,JUMBO BAG RED WHITE SPOTTY,24323.85,93436,0.14,28.62,A,0.69,2.15
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77,5.63,A,0.69,2.84
4750,85123A,CREAM HANGING HEART T-LIGHT HOLDER,155.35,91814,0.00,97.33,C,0.68,3.52
...,...,...,...,...,...,...,...,...,...
855,21409,SPOTTY COCKEREL DOORSTOP,4.25,1,0.00,97.33,C,0.00,98.05
3794,72751C,"F.FAIRY S/3 SML CANDLE, ROSE",5.95,1,0.00,97.33,C,0.00,98.05
4904,90000C,OPAL WHITE/SILVER FLOWER NECKLACE,5.95,1,0.00,97.33,C,0.00,98.05
3792,72751A,"F.FAIRY S/3 SML CANDLE,LILY/VALLEY",5.95,1,0.00,97.33,C,0.00,98.05


In [25]:
def quantity(x):
    if x <=80:
        return 'A'
    elif x<=95:
        return 'B'
    else:
        return 'C'
   # допиши функцию
products['popularity'] = products ['running_percent_quantity'].apply(quantity)


# создай новый столбец popularity с помощью функции

In [26]:
products.head()

,stock_code,description,total,quantity,percent_total,running_total_percent,revenue,percent_quantity,running_percent_quantity,popularity
4031,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,24098.03,105185,0.14,29.60,A,0.77,0.77,A
4727,85099B,RED RETROSPOT JUMBO BAG,9289.35,93436,0.05,57.94,A,0.69,1.46,A
4726,85099B,JUMBO BAG RED WHITE SPOTTY,24323.85,93436,0.14,28.62,A,0.69,2.15,A
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77,5.63,A,0.69,2.84,A
4750,85123A,CREAM HANGING HEART T-LIGHT HOLDER,155.35,91814,0.00,97.33,C,0.68,3.52,A


## Задание 6

__Объедини группы и проанализируй__

1. Создай серию `group` с объединенным столбцом отметок популярности и доходности товаров
2. Посчитай количество вхождений каждой сгруппированной пары
3. Какие группы непопулярны и приносят мало дохода?
4. На каких группах товаров нужно сфокусироваться при составлении плана закупок?

In [27]:
products['group'] = products['revenue']+products['popularity']
products.head()

,stock_code,description,total,quantity,percent_total,running_total_percent,revenue,percent_quantity,running_percent_quantity,popularity,group
4031,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,24098.03,105185,0.14,29.60,A,0.77,0.77,A,AA
4727,85099B,RED RETROSPOT JUMBO BAG,9289.35,93436,0.05,57.94,A,0.69,1.46,A,AA
4726,85099B,JUMBO BAG RED WHITE SPOTTY,24323.85,93436,0.14,28.62,A,0.69,2.15,A,AA
4725,85099B,JUMBO BAG RED RETROSPOT,134307.44,93436,0.77,5.63,A,0.69,2.84,A,AA
4750,85123A,CREAM HANGING HEART T-LIGHT HOLDER,155.35,91814,0.00,97.33,C,0.68,3.52,A,CA


In [28]:
products['group'].value_counts()

CC    2334
AA     765
BB     528
BC     441
CB     433
BA     278
AB     269
CA     157
AC     110
Name: group, dtype: int64

## Задание 7

__Выгрузи данные по товарам с метокой по ABC-анализу__

1. Объедини по `stock_code` в одной таблице `ABC-код`, описание товара и `ID `

2. Выгрузи таблицу в `csv` для отправки в отдел закупок

In [29]:
products = products[['stock_code', 'description', 'group']]
products

,stock_code,description,group
4031,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,AA
4727,85099B,RED RETROSPOT JUMBO BAG,AA
4726,85099B,JUMBO BAG RED WHITE SPOTTY,AA
4725,85099B,JUMBO BAG RED RETROSPOT,AA
4750,85123A,CREAM HANGING HEART T-LIGHT HOLDER,CA
...,...,...,...
855,21409,SPOTTY COCKEREL DOORSTOP,CC
3794,72751C,"F.FAIRY S/3 SML CANDLE, ROSE",CC
4904,90000C,OPAL WHITE/SILVER FLOWER NECKLACE,CC
3792,72751A,"F.FAIRY S/3 SML CANDLE,LILY/VALLEY",CC


In [30]:
products.to_csv('products.csv', index=False)

# 🎉🎉🎉 Поздравляем, ты великолепен! 